In [1]:
import requests
import pandas as pd
import bs4
import os
from lxml import etree
url = "https://movie.douban.com/top250"
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/65.0.3314.0 Safari/537.36 SE 2.X MetaSr 1.0"
}
res = requests.get(url, headers=headers)
html = res.text
xp = etree.HTML(html)

xp

<Element html at 0x14c1ea77d00>

探索xpath用法

In [4]:
from bs4 import BeautifulSoup
def open_url(url):
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/65.0.3314.0 Safari/537.36 SE 2.X MetaSr 1.0"}
    res = requests.get(url, headers=headers)

    return res


res = open_url('https://movie.douban.com/subject/6786002/')
res

<Response [200]>

In [60]:
# 字典合并
dict1 = {}
dict1.update({"1": 2})
dict1.update({"2": 3})
print(dict1)


{'1': 2, '2': 3}


In [16]:
soup = BeautifulSoup(res.text, 'html.parser')  # 用 html.parser 来解析网页
item = soup.find('div', id='content')


In [21]:



movie = {}  # 新建字典，存放电影信息

# 电影名称
movie['电影名称'] = item.h1.span.text

# 导演、类型、制片国家/地区、语言、上映时间、片长（部分电影这些信息不全，先全部采集，留待数据分析时处理）
infos = item.find('div', id='info').text.replace(' ', '').split('\n')
for i in infos:
        if ':' in i:
                movie[i.split(':')[0]] = i.split(':')[1]
        else:
                continue


In [33]:
[i for i in movie.keys()]




['电影名称', '导演', '编剧', '主演', '类型', '制片国家/地区', '语言', '上映日期', '片长', '又名', 'IMDb']

In [26]:
list(movie.values())[1:-1]

['奥利维埃·纳卡什/埃里克·托莱达诺',
 '奥利维埃·纳卡什/埃里克·托莱达诺',
 '弗朗索瓦·克鲁塞/奥玛·希/安娜·勒尼/奥德雷·弗勒罗/托马·索利韦尔',
 '剧情/喜剧',
 '法国',
 '法语',
 '2011-11-02(法国)',
 '112分钟',
 '闪亮人生(港)/逆转人生(台)/无法触碰/最佳拍档/不可触碰/不可触摸/Untouchable/TheIntouchables']

In [11]:
# 导演
xp.xpath(
    '//*[@id="content"]/div/div[1]/ol/li[3]/div/div[2]/div[2]/p[1]/text()')[0].strip().replace("\xa0\xa0\xa0", "\t").split("\t")[1][4:]


'汤姆·汉克斯 Tom Hanks / ...'

In [ ]:
# 导演
director = xp.xpath(
    '//*[@id="content"]/div/div[1]/ol/li[3]/div/div[2]/div[2]/p[1]/text()')[0].strip().replace("\xa0\xa0\xa0", "\t").split("\t")[0][4:]
director
#info
info = xp.xpath(
    '//*[@id="content"]/div/div[1]/ol/li[3]/div/div[2]/div[2]/p[1]/text()')[1].strip().replace("\xa0", "").split("/")
date, area, genre = info[0], info[1], info[2]
genre.replace(" ", ",")

# rate
xp.xpath(
    '//*[@id="content"]/div/div[1]/ol/li[1]/div/div[2]/div[2]/div/span[2]/text()')[0]
# people
xp.xpath(
    '//*[@id="content"]/div/div[1]/ol/li[1]/div/div[2]/div[2]/div/span[4]/text()')[0][:-3]
# quote
xp.xpath(
    '//*[@id="content"]/div/div[1]/ol/li[1]/div/div[2]/div[2]/p[2]/span/text()')[0]
# rank
xp.xpath('//*[@id="content"]/div/div[1]/ol/li[1]/div/div[1]/em/text()')
# 别名

li = '//*[@id="content"]/div/div[1]/ol/li[1]/'
try:
    title2 = xp.xpath(li +
                      'div/div[2]/div[1]/a/span[@class="title"][2]/text()')[0].replace("\xa0", "")
except:

    print(xp.xpath(li +
                   'div/div[2]/div[1]/a/span[2]/text()')[0].replace("\xa0", ""))
else:

    print(title2+xp.xpath(
        li+'div/div[2]/div[1]/a/span[@class="other"]/text()')[0].replace("\xa0", ""))
# 豆瓣链接
xp.xpath(li+'div/div[2]/div[1]/a/@href')[0]

# 图片
xp.xpath('//*[@id="content"]/div/div[1]/ol/li[1]/div/div[1]/a/img/@src')[0]
